In [1]:
import ee
import geemap
# ee.Authenticate()

In [2]:
import sys
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print(e)
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
    return image.addBands([ndvi, gndvi, savi])

Google Earth Engine has initialized successfully!


In [3]:
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
    return image.addBands([ndvi, gndvi, savi])

def create_gif_from_collection(image_collection,
                               clipRegion,
                               bands,
                               min_val,
                               max_val, 
                               palette: ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400']):
    filmArgs = {
    "dimensions": 128,
    "region": clipRegion,
    "crs": 'EPSG:3857',
    "min": min_val, 
    "max" : max_val,
    "palette":  palette,
#     "palette":  ['#006400', '#00c800', '#ffff00', '#ff0000', '#640000'],
#     "palette": ['white', 'green'],
#     "bands": ['B4', 'B3', 'B2'],
    "bands": bands,
    "framesPerSecond": 1, 
}
    return image_collection.map(add_metrics).filterDate('2019-02-01', '2019-06-30').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25).getVideoThumbURL(filmArgs)

def listOfImages_printer(imagesList):
    print(len(imagesList))
    for i in range(len(imagesList)):
        if i < 10:
            print(i,"    ",imagesList[i])
        else: 
            print(i,"   ",imagesList[i])

        if (i+1) % 3 == 0:  
            print(" ")
            
def get_image_date(image): 
    return ee.Date(ee.Image(image).get('system:time_start')).format("yyyy-MM-dd").getInfo()

def show_image(Map, ImagesCollection, index, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    Image_metrics = add_metrics(Image).clip(clipRegion)
    print(get_image_date(Image_metrics))
    Map.addLayer(Image_metrics, params, "", True) 
    return Map

def show_image_diff(Map, ImagesCollection, index_1, index_2, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image_1 = ee.Image(Images.get(index_1)).clip(clipRegion)
    Image_1_metrics = add_metrics(Image_1)

    Image_2 = ee.Image(Images.get(index_2)).clip(clipRegion)
    Image_2_metrics = add_metrics(Image_2)
    print(get_image_date(Image_2), get_image_date(Image_1))
    Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs().select(['NDVI', 'SAVI', 'GNDVI'])
    Map.addLayer(Image_diff, params, "", True) 
    return Map

def get_image(ImagesCollection, index, clipRegion):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    return ee.Image(add_metrics(Image).clip(clipRegion))
    




In [4]:
'''
Region for Kas und Brot
'''
# region = ee.Geometry.Polygon([[[10.821064, 50.428389],
#   [10.818103, 50.427378],
#   [10.817845, 50.424343],
#   [10.816343, 50.423031],
#   [10.817545, 50.421964],
#   [10.817802, 50.419203],
#   [10.818875, 50.41841],
#   [10.820635, 50.419285],
#   [10.819948, 50.419914],
#   [10.82042, 50.422183],
#   [10.822094, 50.421964],
#   [10.823382, 50.421335],
#   [10.823339, 50.420843],
#   [10.823768, 50.42046],
#   [10.828017, 50.424234],
#   [10.82969, 50.424534],
#   [10.83072, 50.42437],
#   [10.831064, 50.425382],
#   [10.823167, 50.427077],
#   [10.821064, 50.428389]]])
'''
Region for Lange
'''
# region = ee.Geometry.Polygon([[[10.816192, 50.399397],
#   [10.817222, 50.398495],
#   [10.816771, 50.39818],
#   [10.817029, 50.397811],
#   [10.816557, 50.397633],
#   [10.81808, 50.396334],
#   [10.81881, 50.395868],
#   [10.820076, 50.39621],
#   [10.82057, 50.395554],
#   [10.820784, 50.395335],
#   [10.822887, 50.395554],
#   [10.820977, 50.3974],
#   [10.820162, 50.397072],
#   [10.819604, 50.397373],
#   [10.82057, 50.397879],
#   [10.819304, 50.399986],
#   [10.819304, 50.400314],
#   [10.817844, 50.400013],
#   [10.816192, 50.399397]]])
'''
Region for Kessel 
'''
# region = ee.Geometry.Polygon([[[10.83377, 50.450762],
#   [10.832998, 50.450161],
#   [10.832848, 50.449724],
#   [10.83347, 50.449696],
#   [10.834693, 50.449369],
#   [10.836216, 50.44915],
#   [10.836966, 50.449218],
#   [10.837803, 50.449068],
#   [10.839777, 50.448357],
#   [10.839712, 50.449095],
#   [10.838747, 50.449478],
#   [10.839004, 50.449669],
#   [10.837052, 50.450147],
#   [10.835143, 50.450421],
#   [10.83377, 50.450762]]]
# )
'''
Region for Kitze

'''
region = ee.Geometry.Polygon([[[10.832673, 50.46425],
  [10.833263, 50.463021],
  [10.834217, 50.462481],
  [10.835033, 50.464032],
  [10.835515, 50.464366],
  [10.835183, 50.464503],
  [10.834604, 50.464394],
  [10.832673, 50.46425]]])
''' 
Region for Langer Acke
'''
# region = ee.Geometry.Polygon([[[10.832573, 50.454615],
#   [10.832069, 50.454041],
#   [10.831296, 50.454089],
#   [10.828314, 50.453774],
#   [10.827531, 50.453788],
#   [10.82532, 50.453535],
#   [10.825728, 50.452101],
#   [10.828496, 50.452155],
#   [10.829333, 50.451916],
#   [10.830932, 50.451718],
#   [10.831951, 50.451882],
#   [10.83429, 50.451643],
#   [10.83503, 50.452203],
#   [10.835041, 50.452893],
#   [10.835212, 50.453071],
#   [10.836178, 50.453843],
#   [10.836103, 50.454041],
#   [10.835255, 50.454369],
#   [10.8337, 50.454539],
#   [10.832573, 50.454615]]])

' \nRegion for Langer Acke\n'

In [5]:
import os

KasUnBrot_collection_2017 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2017-07-01', '2017-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)

listOfImages_2017 = KasUnBrot_collection_2017.aggregate_array('system:index').getInfo()
print(listOfImages_2017)

KasUnBrot_collection_2017_list = KasUnBrot_collection_2017.toList(KasUnBrot_collection_2017.size())


# image = KasUnBrot_collection.first().clip(region)
# image = add_metrics(image)
# out_dir =  '/Users/hungnguyen/Masters/DataScience/DSAssignments/GIS-project/KasUnBrot-11Apr/'
# info = KasUnBrot_collection.aggregate_array('system:index').getInfo()
# filename = out_dir + info[0] + '.tif'
# print(filename)

# geemap.ee_export_image(
#     image, filename=filename, scale=30, region=region, file_per_band=False
# )

['20170706T102021_20170706T102301_T32UPB', '20170719T103021_20170719T103023_T32UPA', '20170719T103021_20170719T103023_T32UPB', '20170808T103021_20170808T103023_T32UPB', '20170815T102021_20170815T102513_T32UPA', '20170815T102021_20170815T102513_T32UPB', '20170825T102021_20170825T102114_T32UPA']


In [6]:
visParams = {"min": 0, 
              "max": 1500, 
              "bands": ["B4", "B3", "B2"]}

ndviParams_diff = {min: 0.2, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['NDVI']}
gndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['GNDVI']}
saviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['SAVI']}

ndviParams = {min: -1, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['NDVI']}




In [6]:
KasUnBrot_collection_2018 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2018-07-01', '2018-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages_2018 = KasUnBrot_collection_2018.aggregate_array('system:index').getInfo()
print(listOfImages_2018)
KasUnBrot_collection_2018_list = KasUnBrot_collection_2018.toList(KasUnBrot_collection_2018.size())






['20180701T102021_20180701T102404_T32UPA', '20180701T102021_20180701T102404_T32UPB', '20180709T103019_20180709T103418_T32UPB', '20180714T103021_20180714T103023_T32UPA', '20180719T103019_20180719T103020_T32UPA', '20180719T103019_20180719T103020_T32UPB', '20180724T103021_20180724T103551_T32UPA', '20180724T103021_20180724T103551_T32UPB', '20180726T102019_20180726T102150_T32UPA', '20180726T102019_20180726T102150_T32UPB', '20180731T102021_20180731T102024_T32UPA', '20180803T103021_20180803T103239_T32UPB', '20180818T103019_20180818T103015_T32UPA', '20180823T103021_20180823T103535_T32UPB']


In [8]:
KasUnBrot_collection_2019 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2019-07-01', '2019-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages_2019 = KasUnBrot_collection_2019.aggregate_array('system:index').getInfo()
print(listOfImages_2019)
KasUnBrot_collection_2019_list = KasUnBrot_collection_2019.toList(KasUnBrot_collection_2019.size())

KasUnBrot_collection_2020 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2020-07-01', '2020-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages_2020 = KasUnBrot_collection_2020.aggregate_array('system:index').getInfo()
print(listOfImages_2020)
KasUnBrot_collection_2020_list = KasUnBrot_collection_2020.toList(KasUnBrot_collection_2020.size())

KasUnBrot_collection_2021 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2021-07-01', '2021-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages_2021 = KasUnBrot_collection_2021.aggregate_array('system:index').getInfo()
print(listOfImages_2021)
KasUnBrot_collection_2021_list = KasUnBrot_collection_2021.toList(KasUnBrot_collection_2021.size())






['20190704T103029_20190704T103317_T32UPA', '20190704T103029_20190704T103317_T32UPB', '20190716T102031_20190716T102028_T32UPA', '20190724T103029_20190724T103030_T32UPA', '20190724T103029_20190724T103030_T32UPB', '20190823T103029_20190823T103244_T32UPA', '20190823T103029_20190823T103244_T32UPB', '20190825T102031_20190825T102430_T32UPA', '20190825T102031_20190825T102430_T32UPB']
['20200723T103031_20200723T103349_T32UPA', '20200730T102031_20200730T102528_T32UPA', '20200730T102031_20200730T102528_T32UPB', '20200807T102559_20200807T103614_T32UPA', '20200807T102559_20200807T103614_T32UPB', '20200812T103031_20200812T103028_T32UPA']
['20210718T103031_20210718T103025_T32UPA', '20210718T103031_20210718T103025_T32UPB', '20210814T102031_20210814T102609_T32UPA', '20210814T102031_20210814T102609_T32UPB']


In [9]:
KasUnBrot_collection_2022 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2022-07-01', '2022-08-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages_2022 = KasUnBrot_collection_2022.aggregate_array('system:index').getInfo()
print(listOfImages_2022)
KasUnBrot_collection_2022_list = KasUnBrot_collection_2022.toList(KasUnBrot_collection_2022.size())




['20220703T102611_20220703T103617_T32UPA', '20220725T101559_20220725T102250_T32UPA', '20220725T101559_20220725T102250_T32UPB', '20220807T102559_20220807T103507_T32UPA', '20220807T102559_20220807T103507_T32UPB', '20220812T103031_20220812T103132_T32UPB']


In [7]:
Images = KasUnBrot_collection_2017_list
listOfImages = listOfImages_2017
print(len(listOfImages))
Image_1 = ee.Image(Images.get(1)).clip(region)
Image_1_metrics = add_metrics(Image_1)

Image_2 = ee.Image(Images.get(-1)).clip(region)
Image_2_metrics = add_metrics(Image_2)

Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs()
print(listOfImages[1], listOfImages[-1])
Map3 = geemap.Map(location=[ 50.422183,10.82], zoom=15)
Map3.addLayer(Image_diff, ndviParams_diff, "MedianImage", True) 
Map3




7
20170719T103021_20170719T103023_T32UPA 20170825T102021_20170825T102114_T32UPA


Map(center=[50.422183, 10.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [28]:
filmArgs = {
    "dimensions": 128,
    "region": region,
    "crs": 'EPSG:3857',
    "min": 0.2, 
    "max" : 1,
    "palette":  ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400'],
#     "palette":  ['#006400', '#00c800', '#ffff00', '#ff0000', '#640000'],
#     "palette": ['white', 'green'],
    "bands": ['NDVI'],
    "framesPerSecond": 1
}


In [13]:
# print(KasUnBrot_collection_2017.map(add_metrics).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10).getVideoThumbURL(filmArgs));
listOfImages_printer(listOfImages_2017)



7
0      20170706T102021_20170706T102301_T32UPB
1      20170719T103021_20170719T103023_T32UPA
2      20170719T103021_20170719T103023_T32UPB
 
3      20170808T103021_20170808T103023_T32UPB
4      20170815T102021_20170815T102513_T32UPA
5      20170815T102021_20170815T102513_T32UPB
 
6      20170825T102021_20170825T102114_T32UPA


In [28]:
Map4 = geemap.Map(location=[50.463021, 10.833263], zoom=15)
show_image(Map4, KasUnBrot_collection_2017, 6, region, visParams)

2017-08-25


Map(center=[50.463021, 10.833263], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [25]:
img_list = [3, 4, 6]

for i in img_list:
    image = get_image(KasUnBrot_collection_2017, i, region)
    field = "Kitze/"
    filename = field + get_image_date(image) + ".tif"
    geemap.ee_export_image(
        image,
        filename=filename,
        scale=30,
        region=region,
        file_per_band=False
)


Generating URL ...
Please wait ...
Data downloaded to /Users/hungnguyen/Masters/DataScience/DSAssignments/GIS-project/Kitze/2017-08-08.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/hungnguyen/Masters/DataScience/DSAssignments/GIS-project/Kitze/2017-08-15.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/hungnguyen/Masters/DataScience/DSAssignments/GIS-project/Kitze/2017-08-25.tif
